In [ ]:
!pip install tensorflow opencv-python-headless matplotlib


In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d google/tinyquickdraw
!unzip tinyquickdraw.zip -d quickdraw_data


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/google/tinyquickdraw
License(s): other
100% 11.1G/11.1G [11:44<00:00, 20.2MB/s]
100% 11.1G/11.1G [11:44<00:00, 16.9MB/s]
Archive:  tinyquickdraw.zip
  inflating: quickdraw_data/baseball bat.ndjson  
  inflating: quickdraw_data/finger.ndjson  
  inflating: quickdraw_data/helmet.ndjson  
  inflating: quickdraw_data/hexagon.ndjson  
  inflating: quickdraw_data/hockey stick.ndjson  
  inflating: quickdraw_data/quickdraw_simplified/The Eiffel Tower.ndjson  
  inflating: quickdraw_data/quickdraw_simplified/The Great Wall of China.ndjson  
  inflating: quickdraw_data/quickdraw_simplified/The Mona Lisa.ndjson  
  inflating: quickdraw_data/quickdraw_simplified/aircraft carrier.ndjson  
  inflating: quickdraw_data/quickdraw_simplified/airplane.ndjson  
  inflating: quickdraw_data/quickdraw_simplified/alarm clock.ndjson  
  inflating: quickdraw_data/quickdraw_simplified/ambulance.ndjson  
  inflating: quickdraw_data/qu

In [ ]:
!pip install ndjson

In [ ]:
import os
import numpy as np
import cv2
import ndjson

def preprocess_quickdraw_ndjson(data_path, output_path, occlusion_probability=0.3):
    incomplete_dir = os.path.join(output_path, "incomplete")
    complete_dir = os.path.join(output_path, "complete")

    os.makedirs(incomplete_dir, exist_ok=True)
    os.makedirs(complete_dir, exist_ok=True)

    for filename in os.listdir(data_path):
        if filename.endswith('.ndjson'):
            with open(os.path.join(data_path, filename)) as f:
                sketches = ndjson.load(f)
            for i, sketch_data in enumerate(sketches):
                sketch = sketch_data['drawing']
                img = np.zeros((256, 256), dtype=np.uint8)

                # Convert sketch to a numpy array and draw it
                for stroke in sketch:
                    points = np.array(list(zip(stroke[0], stroke[1])), dtype=np.int32)
                    cv2.polylines(img, [points], False, 255, 2)

                # Apply random occlusion
                if np.random.rand() < occlusion_probability:
                    occlusion_mask = np.random.randint(0, 2, img.shape).astype(np.uint8)
                    incomplete_img = img * occlusion_mask
                else:
                    incomplete_img = img

                # Save complete and incomplete images
                complete_path = os.path.join(complete_dir, f"{filename.split('.')[0]}_{i}.png")
                incomplete_path = os.path.join(incomplete_dir, f"{filename.split('.')[0]}_{i}.png")
                cv2.imwrite(complete_path, img)
                cv2.imwrite(incomplete_path, incomplete_img)

# Example usage
data_path = "quickdraw_data"  # Path to your .ndjson files
output_path = "preprocessed_data"
preprocess_quickdraw_ndjson(data_path, output_path)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def unet_model(input_shape=(256, 256, 1)):
    inputs = layers.Input(shape=input_shape)

    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    u5 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = Model(inputs=[inputs], outputs=[outputs])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Instantiate the model
model = unet_model()

# Summary of the model
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 256, 256, 64)   │            640 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 256, 256, 64)   │         36,928 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 128, 128, 64)   │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 128, 128, 128)  │        147,584 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 64, 64, 128)    │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 64, 64, 256)    │        295,168 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 64, 64, 256)    │        590,080 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 32, 32, 256)    │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 32, 32, 512)    │      1,180,160 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 32, 32, 512)    │      2,359,808 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose          │ (None, 64, 64, 256)    │        524,544 │ conv2d_7[0][0]         │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 64, 64, 512)    │              0 │ conv2d_transpose[0][0… │
│                           │                        │                │ conv2d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 64, 64, 256)    │      1,179,904 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 64, 64, 256)    │        590,080 │ conv2d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_1        │ (None, 128, 128, 128)  │        131,200 │ conv2d_9[0][0]         │
│ (Conv2DTranspose)    

 Total params: 7,696,193 (29.36 MB)

 Trainable params: 7,696,193 (29.36 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import os

complete_dir = "preprocessed_data/complete"
incomplete_dir = "preprocessed_data/incomplete"

print(f"Number of images in 'complete': {len(os.listdir(complete_dir))}")
print(f"Number of images in 'incomplete': {len(os.listdir(incomplete_dir))}")


Number of images in 'complete': 686210
Number of images in 'incomplete': 686210


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

# Data augmentation configuration
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of the data used for validation
)

def train_model(model, data_dir, batch_size=32, epochs=50):
    # Training data generator
    train_gen = datagen.flow_from_directory(
        data_dir,
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='input',  # The input and output are the same type
        color_mode='grayscale',
        subset='training',
        classes=['incomplete']  # Only look in the 'incomplete' folder
    )

    # Validation data generator
    val_gen = datagen.flow_from_directory(
        data_dir,
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='input',  # The input and output are the same type
        color_mode='grayscale',
        subset='validation',
        classes=['incomplete']  # Only look in the 'incomplete' folder
    )

    # Checkpoint to save the best model
    checkpoint = ModelCheckpoint("unet_model.keras", monitor='val_loss', verbose=1, save_best_only=True)

    # Training the model
    model.fit(
        train_gen,
        steps_per_epoch=len(train_gen),
        validation_data=val_gen,
        validation_steps=len(val_gen),
        epochs=epochs,
        callbacks=[checkpoint]
    )

# Assuming 'model' is your U-Net model already defined
data_dir = "preprocessed_data"  # Directory containing 'complete' and 'incomplete' folders
train_model(model, data_dir)


Found 548968 images belonging to 1 classes.
Found 137242 images belonging to 1 classes.
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  175/17156 ━━━━━━━━━━━━━━━━━━━━ 5:17:26 1s/step - accuracy: 0.5484 - loss: nan

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

def complete_curve_with_model(image_path, model_path="unet_model.h5"):
    model = load_model(model_path)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (256, 256))
    image = np.expand_dims(image, axis=-1)
    image = np.expand_dims(image, axis=0)

    completed_image = model.predict(image)
    completed_image = completed_image.squeeze()

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Incomplete Image")
    plt.imshow(image.squeeze(), cmap='gray')

    plt.subplot(1, 2, 2)
    plt.title("Completed Image")
    plt.imshow(completed_image, cmap='gray')

    plt.show()

# Example usage
image_path = "path_to_your_incomplete_image.png"  # Replace with your image file path
complete_curve_with_model(image_path)
